In [1]:
from net.tokenizer import SubWordEmbReaderUtil
from utils.load import MetaLoader, TrainLoader
from utils.preproces import Preprocessor

In [2]:
swer_path = './sstm_v0p5_deploy/sstm_v4p49_np_n36134_d128.dat'
swer = SubWordEmbReaderUtil(swer_path)

meta_path = '../../datasets/FH_2024/subtask3/mdata.wst.txt.2023.08.23'
meta_loader = MetaLoader(path=meta_path, swer=swer)
img2id, id2img, img_similarity = meta_loader.get_dataset()


<Initialize subword embedding>
loading= ./sstm_v0p5_deploy/sstm_v4p49_np_n36134_d128.dat


In [3]:
train_path = "../../datasets/FH_2024/subtask3/task1.ddata.wst.txt"
train_loader = TrainLoader(path=train_path)
train_dataset = train_loader.get_dataset()

In [4]:
preprocessor = Preprocessor(num_rank=3, num_coordi=4)
train_dataset = preprocessor(train_dataset, img2id, id2img, img_similarity)